In [1]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import requests
import json
from selenium.webdriver import Chrome
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import time
from datetime import datetime

In [2]:
driver = Chrome(executable_path='/Users/chrisjohnson/Downloads/chromedriver')
url = "https://moxy-hotels.marriott.com/our-locations/"
driver.get(url)
time.sleep(30)
# viewhotels_button = driver.find_elements_by_xpath('/html/body/section/pacsys-animation[3]/pacsys-directory/div[3]/button')[0]
# viewhotels_button.click()
# northamhotels_button = driver.find_elements_by_xpath('/html/body/section/pacsys-animation[3]/pacsys-directory/div[2]/pacsys-directory-listings/div/div[5]/a')[0]
# northamhotels_button.click()

In [3]:
page_source = driver.page_source
soup = BeautifulSoup(page_source, 'lxml')

In [4]:
stuff = soup.find_all('a', {'class', 'swell_vertical-xs'})
# stuff
hotel_list = []
for x in stuff:
    hotel = {}
    hotel['name'] = x.text.strip()
    hotel['url'] = x.attrs['href']
    hotel_list.append(hotel)
hotelsurl_df = pd.DataFrame(hotel_list)
hotelsurl_df

,name,url
0,Moxy Taichung,https://www.marriott.com/RMQOX
1,Moxy Tbilisi,https://www.marriott.com/TBSOX
2,Moxy Bandung,https://www.marriott.com/BDOOX
3,Moxy Osaka Honmachi,https://www.marriott.com/OSAOX
4,Moxy Osaka Shin Umeda,https://www.marriott.com/OSAOO
...,...,...
78,Moxy Chattanooga Downtown,https://www.marriott.com/CHAOX
79,Moxy Memphis Downtown,https://www.marriott.com/MEMOX
80,Moxy Nashville Downtown,https://www.marriott.com/BNAOX
81,Moxy Nashville Vanderbilt Area,https://www.marriott.com/BNAOV


In [5]:
%%time

hotel_list = []
counter = 0
for index, row in hotelsurl_df.iterrows():
    name = row['name']
    url = row['url']
    try:
        res = requests.get(url)
        soup = BeautifulSoup(res.content, 'lxml')
        stuff = soup.find('div', {'class' : 'l-container l-header-container'})
#             hotel_list = []
        hotel = {}
        try:
            hotel['brand'] = 'Moxy'
        except:
            hotel['brand'] = 'Moxy'
        try:
            hotel['name'] = name
        except:
            hotel['name'] = name
        try:
            hotel['url'] = url
        except:
            hotel['url'] = url
        try:
            hotel['street'] = stuff.find_all('span')[2].text
        except:
            hotel['street'] = 'N/A'
        try:
            hotel['locality'] = stuff.find_all('span')[4].text
        except:
            hotel['locality'] = 'N/A'        
        try:
            hotel['state'] = stuff.find_all('span')[6].text
        except:
            hotel['state'] = 'N/A'            
        try:
            hotel['postalcode'] = stuff.find_all('span')[7].text
        except:
            hotel['postalcode'] = 'N/A'            
        try:
            hotel['coordinate1'] = stuff.find_all('span')[25].text
        except:
            hotel['coordinate1'] = 'N/A'            
        try:
            hotel['coordinate2'] = stuff.find_all('span')[26].text
        except:
            hotel['coordinate2'] = 'N/A'
        try:
            hotel['coordinate3'] = stuff.find_all('span')[27].text
        except:
            hotel['coordinate3'] = 'N/A'            
        try:
            hotel['coordinate4'] = stuff.find_all('span')[28].text
        except:
            hotel['coordinate4'] = 'N/A'            
        
        desc = soup.find_all('p', {'class' : 'l-margin-top-none l-overflow-hidden t-collapsed-height'})
        try:
            hotel['description'] = str(desc).replace('<p class="l-margin-top-none l-overflow-hidden t-collapsed-height">','').replace('</p>', '')
        except:
            hotel['description'] = 'N/A'
        
        overallratings = stuff = soup.find('div', {'class' : 'l-overall-rating-container'})
        try:
            hotel['overall_rating'] = ratings.find_all('div')[0].text.strip()[:4]
        except:
            hotel['overall_rating'] = 'N/A'
        
        specratings = stuff = soup.find('div', {'class' : 'l-secondary-ratings-container l-clear l-margin-bottom-quarter l-m-margin-bottom-none'})
        try:
            hotel['cleanliness_rating'] = specratings.find_all('div')[2].text.strip()[:4]
        except:
            hotel['cleanliness_rating'] = 'N/A'
        try:
            hotel['service_rating'] = specratings.find_all('div')[7].text.strip()[:4]
        except:
            hotel['service_rating'] = 'N/A'
            
        hotel_list.append(hotel)
        print(hotel)
        print()
        print(datetime.now())
        print('='*80)
        print('='*80)
        print()
#         driver.close()
#         driver.switch_to.window(original_window)
#         time.sleep(1)

        counter += 1
        print(f'Collected {counter} hotels.')
    except:
        hotel = {}
        hotel['name'] = name
        hotel['url'] = url
        hotel_list.append(hotel)
        print(f"Number {x} didn't work at {datetime.now()}")
        continue
hotel_df = pd.DataFrame(hotel_list)
hotel_df         

{'brand': 'Moxy', 'name': 'Moxy Taichung', 'url': 'https://www.marriott.com/RMQOX', 'street': 'No. 288, Wenxin S. 6th Road, Nantun District', 'locality': 'Taichung', 'state': '', 'postalcode': '408', 'coordinate1': 'N/A', 'coordinate2': 'N/A', 'coordinate3': 'N/A', 'coordinate4': 'N/A', 'description': "[ Want to experience 100% Taiwanese style? We've got you!The new global hotel concept? Moxy Taichung brings you more than that. No matter the famous nightlife in Taichung, unique Binlang Beauty, or flashing neon lights, there are all in it. From tradition to innovation, from culture to art, come to Moxy Taichung, we S H O W you all.We embrace all fun hunters to come here. There are cozy guest rooms, smart TVs and showers with high-pressure, as well as various fun-filled public areas and non-stop parties on weekends waiting for you to explore!If you love to play and get a kick out of it, there's no reason to leave. ]", 'overall_rating': 'N/A', 'cleanliness_rating': 'N/A', 'service_rating'

{'brand': 'Moxy', 'name': 'Moxy Seoul Insadong', 'url': 'https://www.marriott.com/SELOX', 'street': '37 Donhwamun-ro 11-gil, Nakwon-dong Jongno-gu', 'locality': 'Seoul', 'state': '', 'postalcode': '03139', 'coordinate1': '126.989306', 'coordinate2': 'N/A', 'coordinate3': 'N/A', 'coordinate4': 'N/A', 'description': "[ Stay in the heart of Seoul at Moxy Seoul Insadong in South Korea. Experience the city's traditional architecture, nightlife, shopping and dining from our boutique lifestyle hotel. Unpack your bags in intelligently designed accommodations with tech-forward features, smart flat-screen TVs, luxury beds and fast free Wi-Fi. Stop at our on-site bar, Seoul at Moxy, to enjoy a welcome drink while taking in a panoramic view of Seoul. When hunger strikes, pick up a snack at our 24/7 grab-and-go restaurant on the 2nd floor. Discover the energy of downtown Seoul while exploring popular attractions including Kyeongbok Place, Myeongdong, Insadong and Iksundong, one of the city's most i

{'brand': 'Moxy', 'name': 'Moxy Paris Charles de Gaulle Airport', 'url': 'https://www.marriott.com/PAROX', 'street': '5 Rue du Signe', 'locality': 'Roissy-en-France', 'state': '', 'postalcode': '95700', 'coordinate1': '2.547397', 'coordinate2': 'N/A', 'coordinate3': 'N/A', 'coordinate4': 'N/A', 'description': "[ The bright and open lobby layout is paired with contemporary furniture. The ''Plug and Meet'' gathering areas provide modern ergonomic seating, large writing walls and 56 inch televisions for presentation projection. The lobby also boasts a state-of-the art video wall and 24/7 Food &amp; Beverage offering a wide selection of snacks and drinks. Other hotel amenities include healthy continental breakfast, an evening bar featuring wines by the glass and local brews. All bedrooms are equipped with 42 inch flat screen televisions, build in USB ports and free Wifi. ]", 'overall_rating': 'N/A', 'cleanliness_rating': '4.4', 'service_rating': '3.7'}

2020-09-21 23:05:56.154728

Collecte

{'brand': 'Moxy', 'name': 'Moxy Frankfurt Eschborn', 'url': 'https://www.marriott.com/FRAOE', 'street': 'Mannheimer Strasse 1', 'locality': 'Eschborn', 'state': '', 'postalcode': '65760', 'coordinate1': 'N/A', 'coordinate2': 'N/A', 'coordinate3': 'N/A', 'coordinate4': 'N/A', 'description': '[ Are you going to the Frankfurt Messe? For business or pleasure? Stay in the Moxy Frankfurt Eschborn; it’s the perfect place to put your feet up after a long day. Moxy is a comfortable hotel with an affordable rate, where you can have a seriously good time. Just the thing you need, with the latest technology and everything at hand. Smart service and stylish design are there to make you feel relaxed. With 24/7 food and drinks at hand, a lively bar with bagels and cappuccinos in the AM, super-duper fast, free Wi-Fi for free, and the free use of computers. We also have meeting areas with a big display to show your presentation. Moxy is just near the station; a short walk and you’re there! The buzzing 

{'brand': 'Moxy', 'name': 'Moxy Frankfurt Airport Kelsterbach', 'url': 'https://www.marriott.com/FRAOK', 'street': 'Am Weiher 1', 'locality': 'Kelsterbach', 'state': '', 'postalcode': '65451', 'coordinate1': 'N/A', 'coordinate2': 'N/A', 'coordinate3': 'N/A', 'coordinate4': 'N/A', 'description': '[ Located next to Frankfurt International Airport destines the Moxy Frankfurt Airport Kelsterbach for the ideal place for both people on the move and those looking to explore the beautiful Rhein-Main area. The MOXY Sleeper and Fabulous Sleeper offer a high level of comfort during your stay with the most essential in-room features like 42-inch flat screen televisions, built in US8 ports and many other fancy gadgets. The bright and open lobby layout is paired with contemporary furnitures and offer our guests a homey and cozy atmosphere during their stay with us. 24/7 food and beverage offerings, a locally inspired bar featuring a wide selection of drinks and snacks, and furiously-fast and free Wi

...

{'brand': 'Moxy', 'name': 'Moxy Denver Cherry Creek', 'url': 'https://www.marriott.com/DENOX', 'street': '240 Josephine Street', 'locality': 'Denver', 'state': 'Colorado', 'postalcode': '80206', 'coordinate1': 'https://www.marriott.com/hotels/maps/travel/denox-moxy-denver-cherry-creek/?directPageRequest=true', 'coordinate2': '39.720275', 'coordinate3': '-104.958153', 'coordinate4': 'N/A', 'description': '[ For an anything-but-ordinary hotel experience, discover Moxy Denver Cherry Creek. From the moment you check-in, which can be celebrated with a free drink at the bar, you will notice the difference in this modern Colorado hotel. Fluid meeting rooms and outdoor event space create an environment ideal for entertaining or creative problem-solving. Recharge in the stylish hotel rooms that feature functional furniture, large flat-screen televisions, cozy chairs, lightning-fast Wi-Fi and eclectic art; many accommodations also boast mountain views. Play games in the living room, indulge in c

{'brand': 'Moxy', 'name': 'Moxy New Orleans Downtown/French Quarter Area', 'url': 'https://www.marriott.com/MSYOF', 'street': "210 O'Keefe Avenue", 'locality': 'New Orleans', 'state': 'Louisiana', 'postalcode': '70112', 'coordinate1': '29.953585', 'coordinate2': '-90.073268', 'coordinate3': 'N/A', 'coordinate4': 'N/A', 'description': "[ Moxy New Orleans Downtown/French Quarter Area is the destination you desire for a getaway to The Big Easy. Cocktail at check-in? We've got you. Lobby bar that's open 24 hours a day to keep the party going throughout the night? We've got you. Incredibly fast, free Wi-Fi for planning your adventures? Yes, we've got that, too. Moxy is your reimagined boutique hotel. Fun-hunters will enjoy expansive rooms, Smart TVs, premier Muk bath amenities and relaxing rainfall showers. Whether traveling for business or pleasure, our 24-hour lobby bar is the hub of all activities and features handcrafted bloody marys and fresh coffee every morning. Throughout the day, y

{'brand': 'Moxy', 'name': 'Moxy NYC Downtown', 'url': 'https://www.marriott.com/NYCOF', 'street': '26 Ann Street', 'locality': 'New York', 'state': 'New York', 'postalcode': '10038', 'coordinate1': 'Reviews', 'coordinate2': ' View Rates ', 'coordinate3': '', 'coordinate4': ' View Rates ', 'description': "[ Pop in and stay a while at Moxy NYC Downtown in Lower Manhattan, where surprises are standard and the unexpected happens every day. Rising above a revitalized downtown in Tribeca, NYC, we offer a bold new hotel experience, brimming with quirky details and cheeky touches. A glowing LED wall welcomes you into our lobby where we've done away with the traditional hotel trappings, opting for a seamless, virtual check-in. Relax in modern accommodations where state-of-the-art amenities like complimentary high-speed Wi-Fi, 50-inch LED TVs and surround sound speakers encourage serious work and play. Recreation, our co-working and play space, defies categorization. It's the buzzy hub of the Ne

{'brand': 'Moxy', 'name': 'Moxy Nashville Downtown', 'url': 'https://www.marriott.com/BNAOX', 'street': '110 3rd Avenue South', 'locality': 'Nashville', 'state': 'Tennessee', 'postalcode': '37201', 'coordinate1': 'Reviews', 'coordinate2': 'https://www.marriott.com/hotels/maps/travel/bnaox-moxy-nashville-downtown/?directPageRequest=true', 'coordinate3': '36.161036', 'coordinate4': '-86.775394', 'description': "[ Attention Fun Hunters: the new Moxy Nashville Downtown invites free spirits and creative thinkers to experience the city like never before, with modern style, seamless service, and an unbeatable location on Broadway. As the only hotel on Honky Tonk Row, we're literally steps away from legendary attractions like the Johnny Cash/Patsy Cline Museum and the Country Music Hall of Fame, as well as Bridgestone Arena, and more. Urban-chic rooms boast exposed brick walls, beds with LED under lighting, complimentary Wi-Fi, and 55-inch HDTVs. Other hotel perks feature locally roasted OSA c

,brand,name,url,street,locality,state,postalcode,coordinate1,coordinate2,coordinate3,coordinate4,description,overall_rating,cleanliness_rating,service_rating
0,Moxy,Moxy Taichung,https://www.marriott.com/RMQOX,"No. 288, Wenxin S. 6th Road, Nantun District",Taichung,,408,N/A,N/A,N/A,N/A,[ Want to experience 100% Taiwanese style? We'...,N/A,N/A,N/A
1,Moxy,Moxy Tbilisi,https://www.marriott.com/TBSOX,Saarbrucken Square,Tbilisi,,0102,44.804822,N/A,N/A,N/A,"[ Plan a stay at Moxy Tbilisi, located in Saar...",N/A,4.8,4.6
2,Moxy,Moxy Bandung,https://www.marriott.com/BDOOX,Jalan Ir H Djuanda No 69,Bandung,,40116,107.612283,N/A,N/A,N/A,[ Moxy Bandung welcomes Fun Hunters to Bandung...,N/A,4.6,4.5
3,Moxy,Moxy Osaka Honmachi,https://www.marriott.com/OSAOX,"2-2-9 Kawaramachi, Chuo-ku",Osaka,,541-0048,https://www.marriott.com/hotels/maps/travel/os...,34.685631,135.5062,N/A,[ Centered amid some of Japan's most dynamic a...,N/A,4.3,3.8
4,Moxy,Moxy Osaka Shin Umeda,https://www.marriott.com/OSAOO,"7-22-1 Fukushima, Fukushima-ku",Osaka,,553-0003,N/A,N/A,N/A,N/A,"[ Located at a prime location in Umeda, one of...",N/A,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78,Moxy,Moxy Chattanooga Downtown,https://www.marriott.com/CHAOX,1220 King Street,Chattanooga,Tennessee,37403,-85.306636,N/A,N/A,N/A,"[ Why hello gorgeous! We’re Moxy Chattanooga, ...",N/A,4.8,4.7
79,Moxy,Moxy Memphis Downtown,https://www.marriott.com/MEMOX,40 North Front Street,Memphis,Tennessee,38103,Reviews,https://www.marriott.com/hotels/maps/travel/me...,35.146567,-90.052744,"[ Get a room at the bar, then get ready to pla...",N/A,4.7,4.4
80,Moxy,Moxy Nashville Downtown,https://www.marriott.com/BNAOX,110 3rd Avenue South,Nashville,Tennessee,37201,Reviews,https://www.marriott.com/hotels/maps/travel/bn...,36.161036,-86.775394,[ Attention Fun Hunters: the new Moxy Nashvill...,N/A,4.7,4.5
81,Moxy,Moxy Nashville Vanderbilt Area,https://www.marriott.com/BNAOV,1911 Belcourt Avenue,Nashville,Tennessee,37212,Reviews,https://www.marriott.com/hotels/maps/travel/bn...,36.136656,-86.799253,[ Relax at our hotel near Music Row at Moxy Na...,N/A,4.6,4.3


In [6]:
hotel_df = hotel_df.drop_duplicates(subset='name')

In [7]:
hotel_df

,brand,name,url,street,locality,state,postalcode,coordinate1,coordinate2,coordinate3,coordinate4,description,overall_rating,cleanliness_rating,service_rating
0,Moxy,Moxy Taichung,https://www.marriott.com/RMQOX,"No. 288, Wenxin S. 6th Road, Nantun District",Taichung,,408,N/A,N/A,N/A,N/A,[ Want to experience 100% Taiwanese style? We'...,N/A,N/A,N/A
1,Moxy,Moxy Tbilisi,https://www.marriott.com/TBSOX,Saarbrucken Square,Tbilisi,,0102,44.804822,N/A,N/A,N/A,"[ Plan a stay at Moxy Tbilisi, located in Saar...",N/A,4.8,4.6
2,Moxy,Moxy Bandung,https://www.marriott.com/BDOOX,Jalan Ir H Djuanda No 69,Bandung,,40116,107.612283,N/A,N/A,N/A,[ Moxy Bandung welcomes Fun Hunters to Bandung...,N/A,4.6,4.5
3,Moxy,Moxy Osaka Honmachi,https://www.marriott.com/OSAOX,"2-2-9 Kawaramachi, Chuo-ku",Osaka,,541-0048,https://www.marriott.com/hotels/maps/travel/os...,34.685631,135.5062,N/A,[ Centered amid some of Japan's most dynamic a...,N/A,4.3,3.8
4,Moxy,Moxy Osaka Shin Umeda,https://www.marriott.com/OSAOO,"7-22-1 Fukushima, Fukushima-ku",Osaka,,553-0003,N/A,N/A,N/A,N/A,"[ Located at a prime location in Umeda, one of...",N/A,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78,Moxy,Moxy Chattanooga Downtown,https://www.marriott.com/CHAOX,1220 King Street,Chattanooga,Tennessee,37403,-85.306636,N/A,N/A,N/A,"[ Why hello gorgeous! We’re Moxy Chattanooga, ...",N/A,4.8,4.7
79,Moxy,Moxy Memphis Downtown,https://www.marriott.com/MEMOX,40 North Front Street,Memphis,Tennessee,38103,Reviews,https://www.marriott.com/hotels/maps/travel/me...,35.146567,-90.052744,"[ Get a room at the bar, then get ready to pla...",N/A,4.7,4.4
80,Moxy,Moxy Nashville Downtown,https://www.marriott.com/BNAOX,110 3rd Avenue South,Nashville,Tennessee,37201,Reviews,https://www.marriott.com/hotels/maps/travel/bn...,36.161036,-86.775394,[ Attention Fun Hunters: the new Moxy Nashvill...,N/A,4.7,4.5
81,Moxy,Moxy Nashville Vanderbilt Area,https://www.marriott.com/BNAOV,1911 Belcourt Avenue,Nashville,Tennessee,37212,Reviews,https://www.marriott.com/hotels/maps/travel/bn...,36.136656,-86.799253,[ Relax at our hotel near Music Row at Moxy Na...,N/A,4.6,4.3


In [8]:
hotel_df.to_csv('moxy_description.csv')